In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gym
from gym import wrappers
import matplotlib.pyplot as plt
import random
from IPython import display
import time
%matplotlib inline
float_formatter = lambda x: "%.4f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
pd.options.display.float_format = '{:20,.3f}'.format

In [ ]:
env = gym.make('Taxi-v2')
# env = gym.make('FrozenLake-v0')

In [ ]:
def getP(C):
    P = C/np.sum(C,axis=2)[:,:,np.newaxis]
    return(P)

In [ ]:
V = np.zeros(env.observation_space.n)
C = np.zeros([env.action_space.n,env.observation_space.n,env.observation_space.n])+0.01
R = np.zeros([env.action_space.n,env.observation_space.n,env.observation_space.n])
Pi = np.random.randint(env.action_space.n,size = (env.observation_space.n))
P = getP(C)
# P = np.zeros([env.observation_space.n,env.observation_space.n,])+1/(env.observation_space.n)

In [ ]:
episodeMax = 1500
gamma = 0.99
done = False
weight = 0.95

rList = []
avgRList = []
flag = True

for episode in range(episodeMax):
    state = env.reset()
    done = False
    while not done:
        statePre = state
        a = Pi[statePre]
        state,reward,done,info = env.step(a)
        R[a,statePre,state] =  R[a,statePre,state]*(1-weight) + reward*weight
        C[a,statePre,state] += 1
        P = getP(C)
        
        # evaluate the current policy
        V[statePre] = V[statePre]*(1-weight)+(weight)*np.dot(P[a,statePre,:],(R[a,statePre,:]+V*gamma).T)
        
        # improve policy
#         QBest = V
        Qsa =np.einsum('ijk,ijk->ji',P[:,:,:],R[:,:,:]+V[np.newaxis,np.newaxis,:]*gamma)
        Pi = np.argmax(Qsa+0.05*np.random.rand(env.observation_space.n,env.action_space.n),axis = 1)
        
    if done:
        rList.append(reward)
    
    if episode >100:
        avgR = sum(rList[-100:])/100
        avgRList.append(avgR)
        if avgR>0.70 and flag:
            flag = False
            print(episode)

In [ ]:
plt.plot(avgRList)